In [ ]:
import os
import numpy as np

from read_data import ReadClass

experiment_name = "exRo9N10e01_LES"
figure_directory = "../figure/" + experiment_name
if not os.path.isdir(figure_directory):
    os.makedirs(figure_directory)
reader = ReadClass(experiment_name)
reader.read_geometry(0)
reader.read_budget(0)
time_period = reader.time / reader.vortex_period
time_days = reader.time / 86400
total_energy = reader.KE + reader.PE

output_per_day = 86400 / reader.output_interval

NT = np.size(time_period)

In [ ]:
DKE = reader.DKE / reader.output_interval
DPE = reader.DPE / reader.output_interval
Gamma = DPE / DKE
Gamma_cum = reader.DPE / reader.DKE

total_dissipation = DKE + DPE

PKE = reader.PKE / reader.output_interval
PPE = reader.PPE / reader.output_interval
total_production = PKE + PPE

# Running mean
num = round(reader.vortex_period / reader.output_interval)
print(num)
b = np.ones(num)/num
total_production_mean = np.convolve(total_production, b, mode='same')
KE_mean = np.convolve(reader.KE, b, mode='same')
PE_mean = np.convolve(reader.PE, b, mode='same')
PWE_mean = np.convolve(reader.PWE, b, mode='same')
VE_mean = np.convolve(reader.VE, b, mode='same')
DKE_mean = np.convolve(DKE, b, mode='same')
DPE_mean = np.convolve(DPE, b, mode='same')
total_energy_mean = np.convolve(total_energy, b, mode='same')

production_time_scale = total_energy_mean / total_production_mean
dissipation_time_scale = total_energy_mean / DKE_mean

Ozmidov_scale = np.sqrt(total_dissipation / reader.buoyancy_frequency**3)

KE_max = np.max(KE_mean)
PE_max = np.max(PE_mean)
DKE_max = np.max(DKE_mean)
DPE_max = np.max(DPE_mean)
print(KE_max, PE_max, DKE_max, DPE_max)

In [ ]:
KE_H_mean, PE_H_mean = reader.horizontam_mean_energy_series(0, NT)
KE_H_mean_mean = np.convolve(KE_H_mean, b, mode='same')
PE_H_mean_mean = np.convolve(PE_H_mean, b, mode='same')

In [ ]:
num = round(reader.vortex_period / reader.output_interval) 
print(num)
b = np.ones(num)/num

PW1 = reader.PW1 / reader.output_interval
PW2 = reader.PW2 / reader.output_interval
PVE = reader.PVE / reader.output_interval

PW1_mean = np.convolve(PW1, b, mode='same')
PW2_mean = np.convolve(PW2, b, mode='same')
PVE_mean = np.convolve(PVE, b, mode='same')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ptick
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


growth_rate = 0.00704 * 1.e-4
theoretical_series = np.exp(growth_rate * reader.time * 2) * 1.8e-8

rc('text', usetex=True)
rc('font', family='serif')
fs = 14
plt.rcParams["font.size"] = 12

fig = plt.figure(figsize=(6, 7))
fig.subplots_adjust(left=0.15, bottom=0.08, right=0.97,
                    top=0.95, wspace=0.2, hspace=0.25)

ax1 = fig.add_subplot(211)
ax1.plot(time_days, total_energy_mean, 'k', label=r'$E_u + E_\theta$')
ax1.plot(time_days, KE_mean, 'r', label=r'$E_u$')
ax1.plot(time_days, PE_mean, 'b', label=r'$E_\theta$')
ax1.plot(time_days, KE_H_mean_mean, 'm', label=r'$E_u$ (horizontal mean)')
ax1.plot(time_days, theoretical_series, 'gray', linestyle='--')

grid_x_ticks_minor = np.arange(0, 260, 10)
grid_x_ticks_major = np.arange(0, 300, 50)
grid_y_ticks_minor = np.arange(0, 2.6e-4, 0.1e-4)
grid_y_ticks_major = np.arange(0, 3.e-4, 0.5e-4)

ax1.set_xticks(grid_x_ticks_minor, minor=True)
ax1.set_xticks(grid_x_ticks_major, major=True)
ax1.set_yticks(grid_y_ticks_minor, minor=True)
ax1.set_yticks(grid_y_ticks_major, major=True)
ax1.set_xlim(0, 250);
ax1.set_ylim(0, 2.5e-4)
ax1.grid()

ax1.set_xlabel(r'$t$ [days]', fontsize=fs)
ax1.set_ylabel(r'Energy density [J/kg]', fontsize=fs)
ax1.legend(fontsize=fs)

ax1.yaxis.set_major_formatter(ptick.ScalarFormatter(useMathText=True)) 
ax1.yaxis.offsetText.set_fontsize(10)
ax1.ticklabel_format(style='sci',axis='y',scilimits=(0,0))
fig.text(0.05, 0.95, r"(a)", fontsize=fs)

ax2 = fig.add_subplot(212)
ax2.plot(time_days, DKE_mean, 'r', label=r'$\epsilon_u$')
ax2.plot(time_days, DPE_mean, 'b', label=r'$\epsilon_\theta$')

grid_x_ticks_minor = np.arange(0, 260, 10)
grid_x_ticks_major = np.arange(0, 300, 50)
grid_y_ticks_minor = np.arange(0, 2.1e-10, 0.1e-10)
grid_y_ticks_major = np.arange(0, 2.5e-10, 0.5e-10)

ax2.set_xticks(grid_x_ticks_minor, minor=True)
ax2.set_xticks(grid_x_ticks_major, major=True)
ax2.set_yticks(grid_y_ticks_minor, minor=True)
ax2.set_yticks(grid_y_ticks_major, major=True)
ax2.set_xlim(0, 250);
ax2.set_ylim(0, 2.e-10)
ax2.grid()

ax2.set_xlabel(r'$t$ [days]', fontsize=fs)
ax2.set_ylabel(r'Dissipation rate [W/kg]', fontsize=fs)
ax2.legend(fontsize=fs, loc=2)

axins = inset_axes(ax2, width=2.0, height=1.2, loc=1, borderpad=1)
axins.plot(time_days, DKE_mean, 'r', label=r'$\epsilon_u$')
axins.plot(time_days, DPE_mean, 'b', label=r'$\epsilon_\theta$')

grid_x_ticks_minor = np.arange(0, 260, 10)
grid_x_ticks_major = np.arange(0, 300, 50)

axins.set_xticks(grid_x_ticks_minor, minor=True)
axins.set_xticks(grid_x_ticks_major, major=True)
axins.set_xlim(0, 250);
axins.set_ylim(1.e-16, 5.e-10)
axins.set_yscale('log');
axins.grid()
fig.text(0.05, 0.475, r"(b)", fontsize=fs)

figure_name = "energy_time_series_" + experiment_name
fig.savefig(figure_directory + "/" + figure_name + '.eps')
fig.savefig(figure_directory + "/" + figure_name + '.png')
time_days[-1]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ptick

growth_rate = 0.00704 * 1.e-4
theoretical_series = np.exp(growth_rate * reader.time * 2) * 1.8e-8

rc('text', usetex=True)
rc('font', family='serif')
fs = 14
plt.rcParams["font.size"] = 12

fig = plt.figure(figsize=(6, 7))
fig.subplots_adjust(left=0.15, bottom=0.08, right=0.97,
                    top=0.95, wspace=0.2, hspace=0.25)

ax1 = fig.add_subplot(211)
ax1.plot(time_days, total_energy, 'k', label=r'$E_u + E_\theta$')
ax1.plot(time_days, reader.KE, 'r', label=r'$E_u$')
ax1.plot(time_days, reader.PE, 'b', label=r'$E_\theta$')
ax1.plot(time_days, theoretical_series, 'gray', linestyle='--')

grid_x_ticks_minor = np.arange(0, 121, 1)
grid_x_ticks_major = np.arange(0, 130, 10)

ax1.set_xticks(grid_x_ticks_minor, minor=True)
ax1.set_xticks(grid_x_ticks_major, major=True)
ax1.set_xlim(0, 100);
ax1.set_ylim(1.e-9, 1.e-3)

ax1.grid()
ax1.set_yscale('log')
ax1.set_xlabel(r'$t$ [days]', fontsize=fs)
ax1.set_ylabel(r'Energy density [J/kg]', fontsize=fs)
ax1.legend(fontsize=fs, ncol=2)

ax2 = fig.add_subplot(212)
ax2.plot(time_days, DKE_mean, 'r', label=r'$\epsilon_u$')
ax2.plot(time_days, DPE_mean, 'b', label=r'$\epsilon_\theta$')

grid_x_ticks_minor = np.arange(0, 121, 1)
grid_x_ticks_major = np.arange(0, 130, 10)
grid_y_ticks_minor = np.arange(0, 5.1e-10, 0.1e-10)
grid_y_ticks_major = np.arange(0, 6.e-10, 1.e-10)

ax2.set_xticks(grid_x_ticks_minor, minor=True)
ax2.set_xticks(grid_x_ticks_major, major=True)
ax2.set_yticks(grid_y_ticks_minor, minor=True)
ax2.set_yticks(grid_y_ticks_major, major=True)
ax2.set_xlim(0, 120);
ax2.set_ylim(0, 5.e-10)
ax2.grid()
ax2.set_xlabel(r'$t$ [days]', fontsize=fs)
ax2.set_ylabel(r'Dissipation rate [W/kg]', fontsize=fs)
ax2.legend(fontsize=fs)
fig.text(0.05, 0.475, r"(b)", fontsize=fs)

figure_name = "energy_time_series_log" + experiment_name
fig.savefig(figure_directory + "/" + figure_name + '.eps')
fig.savefig(figure_directory + "/" + figure_name + '.png')
time_days[-1]